<a href="https://colab.research.google.com/github/insightcampus/202008-youth-bigdata/blob/master/bokyoung/%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81_%EC%8B%A4%EC%8A%B52_LDA_%EC%98%81%EC%96%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from sklearn.datasets import fetch_20newsgroups

import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
#뉴스 다운로드 및 전처리
def get_news(apply_split=True):
  #20newsgroup 다운로드
  dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers','quotes'))
  documents = dataset.data

  news_df= pd.DataFrame({'document':documents})

  #전처리 
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]"," ") #특수 문자 제거
  news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
  news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower()) # 전체 단어에 대한 소문자 변환

  tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) #토큰화

  stop_words = stopwords.words('english') # NLTK 불용어 조회 
  
  if apply_split:
    return tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
  else:
    return tokenized_doc.apply(lambda x: ' '.join([item for item in x if item not in stop_words]))

In [11]:
#공백으로 토큰 분리
def my_tokenizer(text):
  return text.split()

tokenized_docs = get_news(False)

In [12]:
tokenized_docs

0        well sure story seem biased disagree statement...
1        yeah expect people read actually accept hard a...
2        although realize principle strongest points wo...
3        notwithstanding legitimate fuss proposal much ...
4        well change scoring playoff pool unfortunately...
                               ...                        
11309    danny rubenstein israeli journalist speaking t...
11310                                                     
11311    agree home runs clemens always memorable kinda...
11312    used deskjet orange micros grappler system upd...
11313    argument murphy scared hell came last year han...
Name: clean_doc, Length: 11314, dtype: object

In [13]:
type(tokenized_docs)

pandas.core.series.Series

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tfidf_vect = TfidfVectorizer(tokenizer = my_tokenizer)
tfidf = tfidf_vect.fit_transform(tokenized_docs)
lda= LatentDirichletAllocation(n_components=20)

lda_output= lda.fit_transform(tfidf)

In [16]:
#!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 17.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=c467c0b56676663977075da2d7bc3b925cf9f88dfedf619bd25df45e8d94f624
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=fad92088d251c636d4d392b0b8cd352830e25b86a8af6fc9da37a1a8f5219c10
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [17]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne')
pyLDAvis.display(vis)